In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
cd "/content/drive/MyDrive"

/content/drive/MyDrive


In [33]:
from sentence_transformers import SentenceTransformer, losses
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import InputExample
from torch.utils.data import DataLoader
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import random
random.seed(42)

## 행민 데이터(2800개만)

In [9]:
df_public = pd.read_excel("행정안전부_민원안내정보_20250327.xlsx")[:2800]

In [15]:
# 청크 생성
documents = []
metadatas = []
for idx, row in df_public.iterrows():
    # chunk = "\n".join([f"[{col}] {row[col]}" for col in chunks_in])
    chunk = f"'{row['사무명']}'은 {row['사무개요']} \n 이 민원은 {row['소관부처']} {row['담당부서']}에서 담당하고 있습니다."
    documents.append(chunk)
print(documents[0])
df_public["chunk"] = documents

'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다. 
 이 민원은 국토교통부 공간정보제도과에서 담당하고 있습니다.


## GPT-4o mini QA생성 데이터

In [16]:
querey_df = pd.read_excel("0_2799_질의응답데이터셋.xlsx")
querey_df.head()

,사무명,질문,답변
0,토지임야대장등본교부 다량신청,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,안녕하세요! 여러 필지의 토지임야대장등본이 필요하시군요. 이 민원은 '토지임야대장등...
1,토지임야대장등본교부 다량신청,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...","토지임야대장등본을 한 번에 신청하고자 하신다면, 다음과 같은 정보를 참고하시면 됩니..."
2,토지임야대장등본교부 다량신청,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",상속받은 토지의 등본을 다량으로 신청하시는 것에 대해 안내해 드리겠습니다.\n\n먼...
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...","토지임야대장등본을 다량으로 신청하시려면, 인터넷으로 신청이 가능합니다. 이 민원은 ..."
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...","여러 필지의 토지임야대장등본을 한 번에 신청하시려는군요! 이 민원은 ""토지임야대장등..."


## QA와 행민 데이터 결합

In [17]:
# 병합 (Join)
df_merged = pd.merge(df_public, querey_df, on='사무명', how='left')

print(df_merged.shape)
df_merged.head()

(14000, 18)


,분류번호,사무명,사무개요,신청유형,신청자 자격,수수료,수수료내역,소관부처,담당부서,접수방법,접수/처리,근거법령,신청서,발급물 내용,구비서류,chunk,질문,답변
0,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,안녕하세요! 여러 필지의 토지임야대장등본이 필요하시군요. 이 민원은 '토지임야대장등...
1,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...","토지임야대장등본을 한 번에 신청하고자 하신다면, 다음과 같은 정보를 참고하시면 됩니..."
2,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",상속받은 토지의 등본을 다량으로 신청하시는 것에 대해 안내해 드리겠습니다.\n\n먼...
3,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...","토지임야대장등본을 다량으로 신청하시려면, 인터넷으로 신청이 가능합니다. 이 민원은 ..."
4,SG4CADM-2015,토지임야대장등본교부 다량신청,토지임야대장등본교부 민원에 대하여 여러 번지를 한번에 신청할 수 있는 민원입니다.,교부,누구나신청가능,NaN,NaN,국토교통부,공간정보제도과,인터넷,NaN,NaN,NaN,NaN,NaN,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...","여러 필지의 토지임야대장등본을 한 번에 신청하시려는군요! 이 민원은 ""토지임야대장등..."


In [18]:
chunks_in = ["사무명","질문", "chunk"]
df = df_merged[chunks_in]
df.head()

,사무명,질문,chunk
0,토지임야대장등본교부 다량신청,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
1,토지임야대장등본교부 다량신청,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
2,토지임야대장등본교부 다량신청,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...


### 네거티브 청크 - 하드 네거티브 마이닝

In [22]:
# 임베딩을 생성하는 함수
def get_embeddings(df):
    # 질문과 청크에 대한 임베딩 생성
    anchor_embeddings = model.encode(df['질문'].tolist(), convert_to_tensor=True)
    positive_embeddings = model.encode(df['chunk'].tolist(), convert_to_tensor=True)
    return anchor_embeddings, positive_embeddings

# 하드 네거티브 마이닝 함수
def hard_negative_mining(df, anchor_embeddings, positive_embeddings):
    negative_chunks = []

    for idx, row in df.iterrows():
        # 현재 질문의 임베딩
        anchor_embedding = anchor_embeddings[idx]

        # 모든 청크 간의 코사인 유사도 계산
        cosine_scores = util.pytorch_cos_sim(anchor_embedding, positive_embeddings)[0]

        # 현재 질문과 같은 사무명의 청크는 제외
        same_saemung = df[df['사무명'] == row['사무명']].index
        cosine_scores[same_saemung] = -np.inf  # 같은 사무명의 청크는 제외 (유사도 매우 낮게 설정)

        # 하드 네거티브 (가장 유사한 청크 선택)
        hardest_negative_idx = torch.argmax(cosine_scores).item()

        # 하드 네거티브 청크 선택
        negative_chunks.append(df.iloc[hardest_negative_idx]['chunk'])

    return negative_chunks

model = SentenceTransformer('jhgan/ko-sroberta-multitask')
anchor_embeddings, positive_embeddings = get_embeddings(df)
negative_chunks = hard_negative_mining(df, anchor_embeddings, positive_embeddings)
df['negative_chunk'] = negative_chunks

df.head()

<ipython-input-22-d4b1316d91b8>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['negative_chunk'] = negative_chunks


,사무명,질문,chunk,negative_chunk
0,토지임야대장등본교부 다량신청,1. 최근에 농사를 짓기 위해 여러 필지의 토지임야대장등본이 필요합니다. 여러 번지...,'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,'농업경영체 등록(변경등록) 확인서 발급 신청'은 o 농업경영체 등록 확인서 : 농...
1,토지임야대장등본교부 다량신청,"2. 부동산 개발 사업을 진행하고 있는데, 여러 필지의 토지임야대장등본을 한 번에 ...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,'토지이용계획확인신청'은 토지에 대한 도시계획의 결정사항 및 도시계획구역내의 행위의...
2,토지임야대장등본교부 다량신청,"3. 상속받은 토지의 등본을 다량으로 신청해야 하는데, 각 필지에 대한 정보를 어떻...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,'토지이용계획확인신청'은 토지에 대한 도시계획의 결정사항 및 도시계획구역내의 행위의...
3,토지임야대장등본교부 다량신청,"4. 토지임야대장등본을 다량으로 신청하려고 하는데, 인터넷으로도 신청이 가능한가요?...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,'건축물대장 등본(초본) 발급(열람)'은 이 민원은 건축물대장의 발급 또는 열람을 ...
4,토지임야대장등본교부 다량신청,"5. 여러 필지의 토지임야대장등본을 한 번에 신청하려는데, 신청 수수료는 어떻게 되...",'토지임야대장등본교부 다량신청'은 토지임야대장등본교부 민원에 대하여 여러 번지를 한...,"'공유토지분할에 관한 서류의 열람, 등본발급'은 공유토지(건물이 있는경우에 한함) ..."


### 네거티브 청크 - 랜덤 샘플링

In [ ]:
# # negative_chunk를 랜덤으로 선택
# def assign_negative_chunk(df):
#     negative_chunks = []

#     for idx, row in df.iterrows():
#         now = row['사무명']

#         # 사무명이 겹치지 않는 chunk
#         available_chunks = df[df['사무명'] != now]['chunk'].tolist()

#         # 랜덤으로 하나의 청크를 선택
#         negative_chunk = random.choice(available_chunks)
#         negative_chunks.append(negative_chunk)

#     df['negative_chunk'] = negative_chunks
#     return df

# # negative_chunk를 할당한 데이터프레임 생성
# df = assign_negative_chunk(df_merged)

In [34]:
# 트리플렛 데이터셋을 생성
train_data = []
for i in range(len(df)):
    example = InputExample(
        texts=[df.loc[i, "질문"], df.loc[i, "chunk"], df.loc[i, "negative_chunk"]]
    )
    train_data.append(example)

train_dataloader = DataLoader(train_data, batch_size=16)
train_loss = losses.TripletLoss(model=model)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

In [37]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=5,
    warmup_steps=100,
    show_progress_bar=True,
    output_path="embeddingModel/fine_tuned_model"
)

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: omsluck7 (omsluck7-kwangwoon-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,1.792900
1000,1.105600
1500,0.784500
2000,0.614200
2500,0.502800
3000,0.362300
3500,0.326600
4000,0.211700


In [39]:
model.save("fine_tuned_model")

In [40]:
# 저장된 모델 불러오기
model2 = SentenceTransformer("fine_tuned_model")

# 예시: 문장 임베딩 확인
sentences = ["예시 문장입니다."]
embeddings = model2.encode(sentences)
print(embeddings)

[[ 1.67358860e-01 -5.33908188e-01  6.24536812e-01  3.29823971e-01
  -4.06003445e-01 -8.31151307e-01  2.49873742e-01 -1.09989829e-01
  -1.20890312e-01 -5.40632010e-01  4.85031217e-01  1.82798699e-01
  -1.42322406e-01 -1.59188569e-01  7.66460896e-02 -3.76783349e-02
  -5.39649189e-01 -6.54327869e-01  2.24748120e-01  1.39919803e-01
  -5.44845045e-01  7.59196103e-01  1.68213800e-01 -5.42232133e-02
  -7.82614127e-02  8.90402496e-01  5.40400326e-01 -8.28861147e-02
   5.47479326e-03 -4.35854822e-01  9.90836322e-01 -2.59490341e-01
  -6.75264657e-01  5.99167943e-02 -1.87511191e-01  5.60529292e-01
   1.38653919e-01  1.77023336e-01 -1.56285644e-01 -8.48025262e-01
  -8.42784643e-02  1.56934127e-01  3.57018143e-01  1.14852406e-01
   2.55764097e-01 -5.58788657e-01  5.49622392e-03 -2.91008264e-01
  -1.18173338e-01  1.15197480e-01  2.46917650e-01  1.93763927e-01
  -9.13798809e-01  4.73949522e-01 -5.53269804e-01 -1.89647451e-01
  -3.43739301e-01  3.87529612e-01  2.31517315e-01 -9.75134611e-01
  -2.62274